# Analysis

The main thing we'll go through today is analysis of the Demo Experiment data. We have data from 14 participants who completed the German version of the experiment.

**Before we can start:**

**a**) [Download the data](demo_data.zip) as a `.zip` file.

**b**) Extract the data from the `.zip` file to a folder that makes sense for you.

**c**) Start a new R script in RStudio.

Once everyone has completed these steps, we'll begin...